# Creazione matrici png

In [2]:
import PIL
from PIL import ImageFont
from PIL import Image
from PIL import ImageDraw
import random,string
import os, sys, time

# font = ImageFont.truetype("Arial-Bold.ttf",14)
font = ImageFont.truetype("arial.ttf", 100)

N_SAMPLES = 5000 #numero captcha completi
N_SMP_LETTERE_DEFORMATE =  26000



In [3]:
#creazione cartelle

if not os.path.exists("./matrici/"):
    os.makedirs("./matrici/")
if not os.path.exists("./matrici/originale/"):
    os.makedirs("./matrici/originale/")
if not os.path.exists("./captcha/completi/"):
    os.makedirs("./captcha/completi/")
if not os.path.exists("./captcha/singoli/"):
    os.makedirs("./captcha/singoli/")

In [7]:
size = 200 #px

def creaMatrice(lettera):
    img=Image.new("1", (size,size),color=1)
    draw = ImageDraw.Draw(img)
    draw.text((50, 50),lettera,fill=0,font=font)
    draw = ImageDraw.Draw(img)
    #noisy(img,size)
    img.save(f"./matrici/originale/{lettera}.png")
    


In [17]:
# Funzione di noise per i campioni
def noisy(img,width,height):
    draw = ImageDraw.Draw(img)
    for x in range(0,15):
        draw.point((random.randint(0,width),random.randint(0,height)),0)
        draw.line([(random.randint(0,width),random.randint(0,height)), (random.randint(0,width),random.randint(0,height))],width=3)
        #linee verticali
        xy = (random.randint(0,width),random.randint(0,height))
        xy_due = (random.randint(xy[0]-50,xy[0]+50) ,random.randint(0,height))
        draw.line([xy,xy_due],width=3)
                
    return img

Creazione matrici

In [1]:
#creazione matrici
alfabeto = list(string.ascii_uppercase) #or list(map(chr, range(ord('a'), ord('z')+1)))

#alfabeto = ["A","B","C","D","E"]
for l in alfabeto:
    creaMatrice(l)

NameError: name 'string' is not defined

Augmentor

In [6]:
import Augmentor
p = Augmentor.Pipeline("./matrici/originale/")

Initialised with 26 image(s) found.
Output directory set to ./matrici/originale/output.

In [7]:
p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)

p.random_distortion(probability=1, grid_width=5, grid_height=5, magnitude=8)

p.skew(probability=1)

In [8]:
p.sample(N_SMP_LETTERE_DEFORMATE)

Processing <PIL.Image.Image image mode=1 size=200x200 at 0xED00FD0>: 100%|█| 26000/26000 [06:18<00:00, 68.78 Samples/s] 


# Funzione di add noise per i campioni

In [10]:
def sporcaFolder(path):
    dirs = os.listdir( path )
    for item in dirs:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
            im = noisy(im,width=1000,height=200)
            im.save(f"{f}.png")

In [5]:
import string
def getfirstUppercase(s):
    return ([char for char in s if char.isupper()])[0]

# generazione di captcha random

In [11]:
path = "./matrici/originale/output/" #sono le matrici distorte, 200x200
dirs = os.listdir( path )

MAX_IMAGES=7
MIN_IMAGES=5

max_height = 200

for v in range (0,10):
    num_images = random.randint(MIN_IMAGES,MAX_IMAGES)
    total_width = num_images*200
    
    new_im = Image.new("1", (total_width, max_height))
    filename = ""
    x_offset = 0
    list_images = []
    for i in range (0,num_images): #num_images elementi
        curr_rand=random.randint(0,len(dirs)-1)
        list_images.append(os.path.join(path,dirs[curr_rand]))
        valore = dirs[curr_rand]
        filename+=getfirstUppercase(valore)

    for im in list_images:
        image_cur = Image.open(im)
        new_im.paste(image_cur, (x_offset,0))
        x_offset += image_cur.size[0]
    #sporca new_im
    new_im = noisy(new_im,width=total_width,height=max_height)
    #salva new_im
    new_im.save(f'./captcha/completi/{filename}{v}.png') # es stringa AEDTG45.png

In [13]:
#aggiunge il noise ai captcha
#sporcaFolder("./captcha/completi/")

In [14]:
path = "./captcha/completi/"
dirs = os.listdir( path )

for item in dirs:

    if os.path.isfile(path+item):
        for i in range(0,5):
            im = Image.open(path+item)
            #crop ogni 200px 5 volte
            #0,0 200,200 - 200,0 400,200 - 400,0 600, 200
            current_box = im.crop((200*i,0,200*(i+1),200))

            #legge la lettera corrispondente
            if not os.path.exists("./captcha/singoli/"+item[i]):
                os.makedirs("./captcha/singoli/"+item[i])
            #salva nella cartella corrispondente
            current_box.save(f"./captcha/singoli/{item[i]}/{i}{time.time()}.png")
            


# Generazione di captcha custom

In [18]:
parola = list("AYTHNGGDNF")

save_path = "./CaptchaTest/"

MAX_IMAGES=7
MIN_IMAGES=5
num_images = len(parola)

max_height = 200
total_width = num_images*200


new_im = Image.new("1", (total_width, max_height))
filename = ""
x_offset = 0
list_images = []

matrici = []

for l in parola:
    print(str(l))
    for k in dirs:
        if getfirstUppercase(k) == l: #è buono e lo prendo
            #print(k)
            matrici.append(k)
    curr_rand=random.randint(0,len(matrici)-1)
    list_images.append(path+matrici[curr_rand])
    matrici=[]
            

for im in list_images:
    image_cur = Image.open(im)
    new_im.paste(image_cur, (x_offset,0))
    x_offset += image_cur.size[0]

new_im = noisy(new_im,width=total_width,height=max_height)
new_im.save(f'{save_path}{"".join(parola)}{random.randint(0,500)}.png') # es stringa AEDTG45.png

A
Y
T
H
N
G
G
D
N
F


In [ ]:
#sporca("./CaptchaTest/")